In [1]:
using DataFrames
using Geodesy
using JuMP
using Gurobi
using CSV
using Random
using LinearAlgebra

In [2]:
#Loading historical data file.
df = CSV.read("Dartboard_future_final.csv", normalizenames = true)
# df = df[:,[:FIPS_Code,:Latitude,:Longitude,:Week_Num,:Sales]]
# Loading DC data
dc = CSV.read("Dartboard_DCs.csv", normalizenames = true)
first(dc, 10)
# Selecting only the three existing DCs
# dc = dc[1:3,:]

,Location,Status,Latitude,Longitude,Fixed_Cost,Variable_Cost,Current_Size,Max_Size
,String⍰,Int64⍰,Float64⍰,Float64⍰,Int64⍰,Float64⍰,Int64⍰,Int64⍰
1,Providence,1,41.8,-71.4,0,0.0,1200000,1200000
2,Richmond,1,37.5,-77.4,0,0.0,1200000,1200000
3,Youngstown,1,41.1,-80.6,0,0.0,900000,900000
4,Athens,0,39.3,-82.2,25000000,90.5,0,1200000
5,Baltimore,0,39.3,-76.6,25000000,132.0,0,1200000
6,Bangor,0,44.8,-68.7,25000000,75.0,0,1200000
7,Buffalo,0,42.9,-78.8,25000000,92.2,0,1200000
8,Burlington,0,44.5,-73.1,25000000,148.3,0,1200000
9,Chillicothe,0,39.3,-82.9,25000000,68.7,0,1200000


In [3]:
#Randomly generate sales data for purposes of optimization model. Will get rid of afterwards.
# Random.seed!(15071)
# df[:Sales] = randexp(size(df[1]))
first(df, 10)

,Column1,FIPS_Code,State_Name,County_Name,Latitude,Longitude,Year,Week,Income,Population,Week_Num,Season,Sales
,Int64⍰,Int64⍰,String⍰,String⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64⍰
1,1,9001,Connecticut,Fairfield County,41.244,-73.363,2015,27,1.04926e5,9.48137e5,183,7,1.76609e5
2,2,9001,Connecticut,Fairfield County,41.244,-73.363,2015,28,1.0496e5,9.48157e5,184,7,1.77497e5
3,3,9001,Connecticut,Fairfield County,41.244,-73.363,2015,29,1.04993e5,948178.0,185,8,1.77629e5
4,4,9001,Connecticut,Fairfield County,41.244,-73.363,2015,30,1.05027e5,9.48199e5,186,8,1.78522e5
5,5,9001,Connecticut,Fairfield County,41.244,-73.363,2015,31,1.0506e5,9.48219e5,187,8,1.79419e5
6,6,9001,Connecticut,Fairfield County,41.244,-73.363,2015,32,1.05094e5,948240.0,188,8,1.80321e5
7,7,9001,Connecticut,Fairfield County,41.244,-73.363,2015,33,1.05127e5,9.48261e5,189,9,1.81886e5
8,8,9001,Connecticut,Fairfield County,41.244,-73.363,2015,34,1.05161e5,9.48281e5,190,9,1.82801e5
9,9,9001,Connecticut,Fairfield County,41.244,-73.363,2015,35,1.05194e5,948302.0,191,9,1.8372e5


### Wrangling
We want to optimize over mid-2015 to end-2017, so that includes all data in the file.

We also want to extract data for the projected last 8 weeks of 2017 to gauge estimated peak demand.

In [4]:
df_peak = df[df.Week_Num .>= 305, :]

#Summarize by County
df_county = by(df, [:FIPS_Code, :Latitude, :Longitude], :Sales => sum)
df_peak_county = by(df_peak, [:FIPS_Code, :Latitude, :Longitude], :Sales => sum)
first(df_peak_county, 10)

,FIPS_Code,Latitude,Longitude,Sales_sum
,Int64⍰,Float64⍰,Float64⍰,Float64
1,9001,41.244,-73.363,2.94573e6
2,9003,41.82,-72.718,1.88523e6
3,9005,41.776,-73.202,1.91745e6
4,9007,41.447,-72.529,1.98277e6
5,9009,41.33,-72.927,1.72537e6
6,9011,41.457,-72.127,1.77656e6
7,9013,41.842,-72.308,1.72577e6
8,9015,41.836,-72.02,1.50355e6
9,10001,39.134,-75.448,1.3668e6


In [5]:
##### Defining parameters
num_dc = size(dc,1)
num_counties = size(df_county,1)

#Defining min/max sizes in terms of pallets.
dc_cap_min = dc[:,:Current_Size] * 5 / 13.5
dc_cap_max = dc[:,:Max_Size] * 5 / 13.5

#Define variable costs of building in terms of $ per pallet.
var_cost = dc[:, :Variable_Cost] * 5 /13.5
fixed_cost = dc[:, :Fixed_Cost]

#Define demand in terms of pallets
demand = df_county[:,:Sales_sum] / 1000
demand_peak = df_peak_county[:,:Sales_sum] / 1000

765-element Array{Float64,1}:
 2945.733727439428 
 1885.2315011886121
 1917.4493290599   
 1982.7746021784253
 1725.374407676336 
 1776.5609110399298
 1725.7714369318762
 1503.553934280316 
 1366.7967739837427
 1702.280560772773 
 1529.3078463517227
 2306.697898826291 
 1277.079680984121 
    ⋮              
 1145.9715872197048
 1138.8709165036012
 1296.5410569956339
 1368.7286571287568
 1224.135581172578 
 1117.8620973311552
 1115.432532040416 
  995.536704349491 
 1184.7413548809338
 1084.70624991347  
 1335.2921016270457
 1059.2223917251308

In [6]:
#Distances[i, j] represents the distance from dc i to county j
#We precalculate distances to make easy for optimization.
distances = zeros(num_dc, num_counties)
for i=1:num_dc, j=1:num_counties
    distances[i,j] = distance(LLA(dc[i,:Latitude], dc[i,:Longitude],0.0),
                              LLA(df_county[j,:Latitude], df_county[j,:Longitude],0.0))/1609.34 # meters per mile
end

trans_cost = 1.55/20 # cost per pallet mile

distances

20×765 Array{Float64,2}:
 108.794    68.0662   93.0873   63.335   …  558.691   559.604   610.722
 336.614   388.498   370.117   376.519      240.222   252.418   227.353
 377.24    411.859   386.736   420.529      148.674   137.408   244.293
 485.36    527.683   502.905   530.684       48.3117   37.9978  120.369
 217.334   268.168   247.277   260.86       256.827   262.567   291.327
 340.42    288.567   308.254   301.617   …  762.591   760.121   828.767
 301.9     319.97    296.839   337.018      299.014   289.625   390.897
 225.122   185.992   188.095   212.699      569.293   564.585   645.674
 520.891   562.451   537.579   566.102       84.2254   75.3852  136.562
 184.053   235.981   218.015   224.304      310.233   316.322   339.722
 635.165   661.982   637.655   675.318   …  316.614   301.82    387.175
 673.671   722.181   699.216   718.314      250.748   258.832   172.688
 182.56    131.512   147.242   151.133      598.338   596.265   663.898
 176.643   225.627   203.275   221.663 

In [7]:
### Initialize Model
model = Model(with_optimizer(Gurobi.Optimizer))

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

We define our decision variables:
$$x_{i, j} := \text{Whether DC i is allocated to county j}$$
$$y_{i} := \text{Binary RV signifying whether DC i is built}$$

In [8]:
@variable(model, x[1:num_dc,1:num_counties], Bin);
@variable(model, y[1:num_dc], Bin)

20-element Array{VariableRef,1}:
 y[1] 
 y[2] 
 y[3] 
 y[4] 
 y[5] 
 y[6] 
 y[7] 
 y[8] 
 y[9] 
 y[10]
 y[11]
 y[12]
 y[13]
 y[14]
 y[15]
 y[16]
 y[17]
 y[18]
 y[19]
 y[20]

We define constraints:

1) Each county uses exactly one DC: $\forall j \;\; \sum_{i=1}^{D} x_{i, j} = 1$

2) Ensuring we have week 8 inventory below DC capacity: $\forall i \;\; \sum_{j=1}^{C} d_j \times x_{i, j} \le y_i \times \text{DC_max}_i$


In [9]:
# Each county should use one and only one DC
@constraint(model, nosplit[c=1:num_counties], sum(x[i,c] for i=1:num_dc) == 1)

# Keep peak 8-week period inventory below DC capacityC
@constraint(model, dc_capacity[i=1:num_dc], sum(demand_peak[c]*x[i,c] for c=1:num_counties) <= y[i]*dc_cap_max[i])

#Warehouse can only be allocated if it is built
@constraint(model, y_meaning[i=1:num_dc], sum(x[i, j] for j=1:num_counties) <= num_counties * y[i])

20-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 y_meaning[1] : x[1,1] + x[1,2] + x[1,3] + x[1,4] + x[1,5] + x[1,6] + x[1,7] + x[1,8] + x[1,9] + x[1,10] + x[1,11] + x[1,12] + x[1,13] + x[1,14] + x[1,15] + x[1,16] + x[1,17] + x[1,18] + x[1,19] + x[1,20] + x[1,21] + x[1,22] + x[1,23] + x[1,24] + x[1,25] + x[1,26] + x[1,27] + x[1,28] + x[1,29] + x[1,30] + x[1,31] + x[1,32] + x[1,33] + x[1,34] + x[1,35] + x[1,36] + x[1,37] + x[1,38] + x[1,39] + x[1,40] + x[1,41] + x[1,42] + x[1,43] + x[1,44] + x[1,45] + x[1,46] + x[1,47] + x[1,48] + x[1,49] + x[1,50] + x[1,51] + x[1,52] + x[1,53] + x[1,54] + x[1,55] + x[1,56] + x[1,57] + x[1,58] + x[1,59] + x[1,60] + x[1,61] + x[1,62] + x[1,63] + x[1,64] + x[1,65] + x[1,66] + x[1,67] + x[1,68] + x[1,69] + x[1,70] + x[1,71] + x[1,72] + x[1,73] + x[1,74] + x[1,75] + x[1,76] + x[1,77] + x[1,78] + x[1,79] + x[1,80] + x[1,81] + x[1,82] + x[1,83] + x[1,84] + x[1,85] + x[1,86] + x[1,87] + x[1,88] + x[1,89] + x[1,90] + x[1,91] 

We now define our objective function where the goal is to minimize the total sum of variable and fixed costs:
\begin{align*}
    \text{Min} \;\; \text{delivery costs} &+ \text{building costs} \\
    t \sum_{j=1}^{C} \text{demand}_j \sum_{i=1}^{D} d_{i, j} x_{i, j} &+ \sum_{i=1}^{D} y_i(\text{FC}_i + \text{VC}_i (\sum_{j=1}^{C} d_j x_{i, j}))
\end{align*}


In [10]:
# @objective(model, Min, trans_cost*sum(demand[j]*sum(distances[i,j]*x[i,j] for i=1:num_dc) for j=1:num_counties))
#     + sum(y[i]*(fixed_cost[i] + sum(x[i, j] * demand_peak[j] for j=1:num_counties)) for i=1:num_dc))
# @objective(model, Min, trans_cost * sum(demand[j] * sum(distances[i, j] * x[i, j] for i=1:num_dc) for j=1:num_counties))

@objective(model, Min, dot(demand, transpose(sum(distances .* x, dims = 1)))
    + sum(y[i] * (fixed_cost[i] + sum(x[i, j] * demand_peak[j] for j=1:num_counties)) for i=1:num_dc))

2945.733727439428 y[1]*x[1,1] + 1885.2315011886121 y[1]*x[1,2] + 1917.4493290599 y[1]*x[1,3] + 1982.7746021784253 y[1]*x[1,4] + 1725.374407676336 y[1]*x[1,5] + 1776.5609110399298 y[1]*x[1,6] + 1725.7714369318762 y[1]*x[1,7] + 1503.553934280316 y[1]*x[1,8] + 1366.7967739837427 y[1]*x[1,9] + 1702.280560772773 y[1]*x[1,10] + 1529.3078463517227 y[1]*x[1,11] + 2306.697898826291 y[1]*x[1,12] + 1277.079680984121 y[1]*x[1,13] + 1475.1915028456528 y[1]*x[1,14] + 1557.456152149341 y[1]*x[1,15] + 1291.4771727902837 y[1]*x[1,16] + 1261.7689978441358 y[1]*x[1,17] + 2103.8298738680473 y[1]*x[1,18] + 1481.3457074865853 y[1]*x[1,19] + 1360.8083938412628 y[1]*x[1,20] + 1277.2861445410101 y[1]*x[1,21] + 1408.457745312571 y[1]*x[1,22] + 1273.05311525219 y[1]*x[1,23] + 1265.5949960511991 y[1]*x[1,24] + 1144.586126020079 y[1]*x[1,25] + 1355.112423169769 y[1]*x[1,26] + 1493.696683100037 y[1]*x[1,27] + 1378.66385514509 y[1]*x[1,28] + 1394.6486835445571 y[1]*x[1,29] + 1248.006305975494 y[1]*x[1,30] + 1710.920

In [11]:
optimize!(model)

Academic license - for non-commercial use only
Optimize a model with 805 rows, 15320 columns and 45940 nonzeros
Model has 15300 quadratic objective terms
Variable types: 0 continuous, 15320 integer (15320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+05]
  Objective range  [2e+04, 3e+07]
  QObjective range [2e+03, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5.290366e+09
Presolve removed 20 rows and 0 columns
Presolve time: 0.47s
Presolved: 16085 rows, 30620 columns, 76520 nonzeros
Variable types: 0 continuous, 30620 integer (30620 binary)

Root relaxation: objective 1.132420e+09, 1550 iterations, 0.18 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.1324e+09    0  786 5.2904e+09 1.1324e+09  78.6%     -    1s
H    0     0                    5.282718e+09 1.1324e+09  78.6%     -    1

In [12]:
value.(y)

20-element Array{Float64,1}:
  1.0                   
  1.0                   
  1.0                   
  0.0                   
  1.0                   
  0.0                   
  0.0                   
  0.0                   
  1.0                   
  0.0                   
  0.9999999999999987    
  1.0                   
  1.0                   
  0.0                   
  1.0                   
 -3.2235817957219946e-16
  0.0                   
  1.0                   
  1.0                   
  3.3440737052241285e-16

In [16]:
sizes = zeros(num_dc)
x_val = value.(x)
for i=1:num_dc
    s = sum(x_val[i, j] * demand_peak[j] for j=1:num_counties)
    s = s*13.5/5
    sizes[i] = s
#     print(s)
end
sizes

20-element Array{Float64,1}:
 111363.53534118897         
 266686.39490455127         
 235532.44175158354         
      0.0                   
 324847.5383899424          
      0.0                   
      0.0                   
      9.419960650080046e-12 
 555082.979253575           
      0.0                   
 399284.60573581804         
 299757.2295740601          
 185200.40384342356         
     -8.123239431863957e-11 
 225770.94998914082         
     -3.7100485340396264e-10
     -2.7198543302971032e-11
 189284.11958551488         
 175704.7686771275          
      3.7100485340396264e-10

In [17]:
sizes > 1200000

MethodError: MethodError: no method matching isless(::Int64, ::Array{Float64,1})
Closest candidates are:
  isless(!Matched::Missing, ::Any) at missing.jl:70
  isless(::Real, !Matched::AbstractFloat) at operators.jl:149
  isless(::Integer, !Matched::ForwardDiff.Dual{Ty,V,N} where N where V) where Ty at C:\Users\Abi Shalom\.julia\packages\ForwardDiff\N0wMF\src\dual.jl:140
  ...

In [14]:
@time sum(x_val[i, j] for j=1:num_counties)

  0.193112 seconds (80.69 k allocations: 4.102 MiB, 8.20% gc time)


62.0

Distance obj:
$$t \times \sum_{j=1}^{C} \text{dem}_j \sum_{i=1}^{D} d_{i, j} x_{i, j}$$

In [181]:
##### Loading the historical data file as a dataframe
df = readtable("Dartboard_historical.csv")
# Selecting only the pertinent variables
df = df[:,[:FIPS_Code,:Latitude,:Longitude,:Week_Num,:Sales]]
# Loading DC data
dc = readtable("Dartboard_DCs.csv")
# Selecting only the three existing DCs
dc = dc[1:3,:]

##### Wrangling
# Extracting data for the period 2011 to 2012 only (i.e weeks 1 to 104).
# *** This is a different planning horizon from the one described in the case ***
df_1to104 = df[(df.Week_Num .<= 104),:]
# Extracting data for the last 8 weeks in 2013 - the "peak" demand
df_97to104 = df_1to104[(df_1to104.Week_Num .> 96),:]

# Summarzing but County (while keeping Latitude and Longitude info)
df_1to104_county = by(df_1to104, [:FIPS_Code, :Latitude, :Longitude], :Sales => sum)
df_97to104_county = by(df_97to104, [:FIPS_Code, :Latitude, :Longitude], :Sales => sum)

##### Defining parameters
num_dc = size(dc,1)
num_counties = size(df_1to104_county,1)

# We'll work everything in terms of pallets
# dividing by 1000 converts dollars to pallets
# (You can equivalently work in terms of dollar sales or SQF)

demand_1to104 = df_1to104_county[:,:Sales_sum] / 1000
demand_97to104 = df_97to104_county[:,:Sales_sum] / 1000

dc_cap_min = dc[:,:Current_Size] * 5 / 13.5
dc_cap_max = dc[:,:Max_Size] * 5 / 13.5

distances = zeros(num_dc, num_counties)
for i=1:num_dc, j=1:num_counties
    distances[i,j] = distance(LLA(dc[i,:Latitude], dc[i,:Longitude],0.0),
                              LLA(df_1to104_county[j,:Latitude], df_1to104_county[j,:Longitude],0.0))/1609.34 # meters per mile
end

distances

trans_cost = 1.55/20 # cost per pallet mile

##### Optimization Model

main_mod = Model(with_optimizer(Gurobi.Optimizer));

# The allocation decision of of DCs to counties
@variable(main_mod, x[1:num_dc,1:num_counties], Bin);

# Minimize the sum of transportation costs over the 2012-2013 period
@objective(main_mod, Min, trans_cost*sum(demand_1to104[c]*sum(distances[i,c]*x[i,c] for i=1:num_dc) for c=1:num_counties))

# Each county should use one and only one DC
@constraint(main_mod, nosplit[c=1:num_counties], sum(x[i,c] for i=1:num_dc) == 1)

# Keep peak 8-week period inventory below DC capacityC
@constraint(main_mod, dc_capacity[i=1:num_dc], sum(demand_97to104[c]*x[i,c] for c=1:num_counties) <= dc_cap_max[i])

show(main_mod)

# solve(main_mod)

# println("Objective value: ", getobjectivevalue(mymod))
# Objective value: $137,838,585

# Can also use getvalue(x) to inspect the allocation solution


┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[181]:1
└ @ Core In[181]:1
┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[181]:5
└ @ Core In[181]:5


Academic license - for non-commercial use only
A JuMP Model
Minimization problem with:
Variables: 2295
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.ZeroOne`: 2295 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 765 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 3 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: dc_capacity, nosplit, x

In [110]:
# trans_cost * transpose(demand) * (distances .* x) * ones(num_counties, 1)
# (distances .* x) * ones(num_counties, 1)

In [159]:
# trans_cost * sum(demand[j] * sum(distances[i, j] * x[i, j] for i=1:num_dc) for j=1:num_counties)
demand * sum(distances .* x, dims = 1)

765×765 Array{GenericAffExpr{Float64,VariableRef},2}:
 15.976787239000494 x[1,1] + 49.43289225121889 x[2,1] + 55.39882920719257 x[3,1] + 71.2765971049377 x[4,1] + 31.9162021815787 x[5,1] + 49.991712907282505 x[6,1] + 44.33491652816228 x[7,1] + 33.059900988247286 x[8,1] + 76.49446994789832 x[9,1] + 27.028811843106496 x[10,1] + 93.27599513171405 x[11,1] + 98.93069290921488 x[12,1] + 26.80946210703091 x[13,1] + 25.940554597177258 x[14,1] + 1.4865493141897537 x[15,1] + 22.583959240664598 x[16,1] + 17.162304191810847 x[17,1] + 27.258452037057435 x[18,1] + 78.06207124281076 x[19,1] + 15.973129764635843 x[20,1]    …  89.68640684626219 x[1,765] + 33.38748662195751 x[2,765] + 35.875162764715974 x[3,765] + 17.676511005993618 x[4,765] + 42.782222635510706 x[5,765] + 121.70697356735045 x[6,765] + 57.40438929354184 x[7,765] + 94.8192744417979 x[8,765] + 20.054614399113344 x[9,765] + 49.88921484734467 x[10,765] + 56.85790935271053 x[11,765] + 25.3597620349933 x[12,765] + 97.49553848911304 x[13,765] 

In [168]:
a = [1 2 3; 1 2 3]
b = [1 2 2; 1 1 1]
dem = [1 1 1]
size(demand), size(distances), size(x)
j = 1
sum(a[:, j] .* b[:, j])
j = 765
# sum(distances .* x, 2)
size(sum(a .* b, dims = 1))
# dem * transpose(sum(a .* b, dims = 1))
#Want: [2 6 9]
# sum(demand[j] * sum(distances[:, j] .* x[:, j]) for j=1:num_counties)
# size(sum(transpose(demand[j]) * (distances[:, j] .* x[:, j]) for j=1:num_counties))
# size(transpose(demand)), size(sum(distances .* x, dims = 1))

(1, 3)

In [204]:
trash = transpose(sum(distances .* x, dims = 1))
dot(demand, trash)

15.976787239000494 x[1,1] + 49.43289225121889 x[2,1] + 55.39882920719257 x[3,1] + 71.2765971049377 x[4,1] + 31.9162021815787 x[5,1] + 49.991712907282505 x[6,1] + 44.33491652816228 x[7,1] + 33.059900988247286 x[8,1] + 76.49446994789832 x[9,1] + 27.028811843106496 x[10,1] + 93.27599513171405 x[11,1] + 98.93069290921488 x[12,1] + 26.80946210703091 x[13,1] + 25.940554597177258 x[14,1] + 1.4865493141897537 x[15,1] + 22.583959240664598 x[16,1] + 17.162304191810847 x[17,1] + 27.258452037057435 x[18,1] + 78.06207124281076 x[19,1] + 15.973129764635843 x[20,1] + 8.980449199336142 x[1,2] + 51.25722558668227 x[2,2] + 54.339395953788994 x[3,2] + 69.62092025422226 x[4,2] + 35.38131134194976 x[5,2] + 38.07262619651646 x[6,2] + 42.21593547281461 x[7,2] + 24.53926051980774 x[8,2] + 74.20812064608585 x[9,2] + 31.134615303097615 x[10,2] + 87.33989931670985 x[11,2] + 95.28241152228509 x[12,2] + 17.35126682988721 x[13,2] + 29.76856788310358 x[14,2] + 8.049594122406166 x[15,2] + 13.785462282933745 x[16,2] +

In [183]:
size(demand)

(765,)